In [36]:
import os
import sys
import torch  # Ensure torch is imported
import matplotlib.pyplot as plt
import matplotlib
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import accuracy_score, roc_auc_score
%matplotlib inline
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter


path = os.getcwd()
par_path = os.path.abspath(os.pardir)
sys.path.append(par_path)

from utils.data import ExtractROI, AtlasDataset
from torch.utils.data import Dataset, DataLoader

In [37]:
# if you are running this notebook from scratch, please extract the ROIs and paste the path you want to use
# for train and test
train_dir = '/nafs/dtward/allen/rois/64x64_sample_rate_0_5_center/train/'
# '/nafs/dtward/allen/rois/64x64_sample_rate_0_5_all/train/'
# '/nafs/dtward/allen/rois/64x64_sample_rate_0_5_mode/train/'
# '/nafs/dtward/allen/rois/64x64_sample_rate_0_5_center/train/'

test_dir = '/nafs/dtward/allen/rois/64x64_sample_rate_0_5_center/test/'
# '/nafs/dtward/allen/rois/64x64_sample_rate_0_5_all/test/'
# '/nafs/dtward/allen/rois/64x64_sample_rate_0_5_all/test/'
# '/nafs/dtward/allen/rois/64x64_sample_rate_0_5_center/test/'

# label_map_file converts from parcellation indices to indices 
# for classification (can be one of  divisions.csv, organs.csv, structures.csv, substructures.csv, categories.csv
label_map_file = '/nafs/dtward/allen/rois/divisions.csv' 

train_dataset = AtlasDataset(data_dir=train_dir, label_map_file=label_map_file)
test_dataset = AtlasDataset(data_dir=test_dir, label_map_file=label_map_file)

bs = 16 # this is the batch size. Make it smaller if it takes too long to load a batch.
train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=True)

In [38]:
print(next(iter(train_dataset))[1])
print(train_dataset.num_classes)

[13]
26


In [39]:
# Get a batch of data from the data loader and associate it with the 
'''
    Args:
    data_iter
        passed to the next() function

    Returns:
        the next batch of 'images' and 'labels'
    '''
data_iter = iter(train_loader)
images, labels = next(data_iter)

In [40]:
# Print the shape of the images, this will help know the dimensions of the images and aid in the input for # of channels
print(images.shape)

torch.Size([16, 501, 64, 64])


In [42]:
#Access the label_map from the DataLoader, print out all of the items in the label_map
label_map = test_loader.dataset.label_map
print("Label map:", label_map)

Label map: {0: {'name': 'unassigned', 'index': 0}, 1: {'name': 'HY', 'index': 1}, 27: {'name': 'HY', 'index': 1}, 34: {'name': 'HY', 'index': 1}, 58: {'name': 'HY', 'index': 1}, 67: {'name': 'HY', 'index': 1}, 83: {'name': 'HY', 'index': 1}, 113: {'name': 'HY', 'index': 1}, 121: {'name': 'HY', 'index': 1}, 128: {'name': 'HY', 'index': 1}, 167: {'name': 'HY', 'index': 1}, 187: {'name': 'HY', 'index': 1}, 203: {'name': 'HY', 'index': 1}, 216: {'name': 'HY', 'index': 1}, 219: {'name': 'HY', 'index': 1}, 256: {'name': 'HY', 'index': 1}, 264: {'name': 'HY', 'index': 1}, 276: {'name': 'HY', 'index': 1}, 322: {'name': 'HY', 'index': 1}, 328: {'name': 'HY', 'index': 1}, 337: {'name': 'HY', 'index': 1}, 346: {'name': 'HY', 'index': 1}, 354: {'name': 'HY', 'index': 1}, 380: {'name': 'HY', 'index': 1}, 442: {'name': 'HY', 'index': 1}, 460: {'name': 'HY', 'index': 1}, 505: {'name': 'HY', 'index': 1}, 513: {'name': 'HY', 'index': 1}, 515: {'name': 'HY', 'index': 1}, 604: {'name': 'HY', 'index': 1},

In [24]:
#empty set to store unique names within the label_map
unique_names = set()

# Iterate over the labelmap and collect names
''' 
    Args: 
    label_map.items(): iterating over the dictionary (label_map)
    value.get('name'): retrieval of label's name from 'value' during each iteration

    Return: getting the label index and the values within the dictionary and retrieves all the unique names within 'name'

'''
for key, value in label_map.items():
    name = value.get('name')
    if name:
        unique_names.add(name)

# Print all the unique names we have!
for name in unique_names:
    print(name)

PAL
unassigned
P
TH
MY
Isocortex
lfbs
V4
brain-unassigned
scwm
CB
fiber tracts-unassigned
HPF
CTXsp
eps
cbf
VL
HY
c
V3
cm
AQ
mfbs
OLF
MB
STR


In [25]:

# Extract all 'index' values from the label map
'''
    Args: 
    label_map.items(): retrieving value-pairs from label_map
    Counter(index_list): 

    Return: shows the count of each unique 'index' found in the label_map
'''
index_list = [v['index'] for k, v in label_map.items()]

# Count the occurrences of each unique 'index'
index_counts = Counter(index_list)

# Print out the quantities of all unique 'index' values
print("Quantities of all unique 'index' values in the label_map:")
for index, count in index_counts.items():
    print(f"Index {index}: {count} items")

Quantities of all unique 'index' values in the label_map:
Index 0: 1 items
Index 1: 50 items
Index 2: 241 items
Index 3: 15 items
Index 4: 30 items
Index 5: 59 items
Index 6: 51 items
Index 7: 38 items
Index 8: 17 items
Index 9: 21 items
Index 10: 28 items
Index 11: 9 items
Index 12: 3 items
Index 13: 47 items
Index 14: 21 items
Index 15: 6 items
Index 16: 1 items
Index 17: 11 items
Index 18: 1 items
Index 19: 2 items
Index 20: 22 items
Index 21: 1 items
Index 22: 10 items
Index 23: 1 items
Index 24: 1 items
Index 25: 1 items


In [26]:
def get_indexes_for_name(label_map, target_name):
    '''
    What does this function do?

    Args:
    arg1
        description of arg
        label_map_file is an arguemnt with a dictionary, and the parcellation index value is a key, value contains dictionary and the name

    Returns:
        something
    '''
    indexes = []
    for key, value in label_map.items():
        if value.get('name') == target_name:
            indexes.append(value.get('index'))
    return indexes

# Retrieve index numbers for the name 'brain-unassigned'
unassigned_indexes = get_indexes_for_name(label_map, 'brain-unassigned')

# Print the result
print(f"Indexes for 'brain-unassigned': {unassigned_indexes}")

Indexes for 'brain-unassigned': [23]


In [27]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=501, out_channels=32, kernel_size=3, padding=1, bias=True)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, bias=True)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1, bias=True)
        self.fc1 = nn.Linear(in_features=128*8*8, out_features=256, bias=True)
        self.fc2 = nn.Linear(in_features=256, out_features=num_classes, bias=True)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128*8*8) # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [28]:
# Extract label map from the train_loader
label_map = train_loader.dataset.label_map
num_classes = len(set(v['index'] for v in label_map.values()))

# Initialize the model, criterion, and optimizer
model = SimpleCNN(num_classes=num_classes)

# Compute class weights based on label map for brain_unassigned
weights = torch.ones(num_classes)
for v in label_map.values():
    if v['index'] in [23, 24]:
        weights[v['index']] = 0

criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [29]:
def evaluate_model(loader):
    model.eval()
    all_probs = []
    all_labels = []
    running_loss = 0.0
    with torch.no_grad():
        for data in tqdm(loader, desc='Evaluating', leave=False):
            inputs, labels = data
            labels = labels.long()
            if labels.dim() > 1:
                labels = labels.view(-1)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            probs = torch.softmax(outputs, dim=1)  # Apply softmax to outputs
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Debug: Print shapes and a few samples
    print(f"all_probs shape: {np.array(all_probs).shape}")
    print(f"all_labels shape: {np.array(all_labels).shape}")
    print("Sample all_probs:", all_probs[:5])
    print("Sample all_labels:", all_labels[:5])

    accuracy = accuracy_score(all_labels, np.argmax(all_probs, axis=1))
    
    try:
        # Compute ROC AUC score using one-vs-one (ovo) strategy
        auc = roc_auc_score(all_labels, all_probs, multi_class='ovo', labels=torch.arange(num_classes).numpy())
    except ValueError:
        print("ValueError: AUC calculation failed. Returning 0.")
        auc = 0  # In case there is an error with AUC computation

    return accuracy, auc, running_loss / len(loader)

In [30]:
# Lists to store the metrics
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []
train_aucs = []
test_aucs = []

In [31]:
# Train the model
'''
    args:
    num_epochs: total number of epochs for model training, loop iterates over 
    desc=f'Epoch: descripes the progress of each epoch

'''
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for data in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=True):
        inputs, labels = data
        labels = labels.long()
        # Ensure labels are 1D
        if labels.dim() > 1:
            labels = labels.view(-1)
        # Debugging: print shapes and types
        print(f"Input shape: {inputs.shape}, Input type: {inputs.dtype}")
        print(f"Labels shape: {labels.shape}, Labels type: {labels.dtype}")
        
        optimizer.zero_grad()
        outputs = model(inputs)
        print(f"Outputs shape: {outputs.shape}, Outputs type: {outputs.dtype}")  # Debugging: print shape and type of outputs
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_losses.append(running_loss / len(train_loader))

Epoch 1/10:   1%|          | 1/100 [00:00<00:26,  3.71it/s]

Input shape: torch.Size([16, 501, 64, 64]), Input type: torch.float32
Labels shape: torch.Size([16]), Labels type: torch.int64
Outputs shape: torch.Size([16, 26]), Outputs type: torch.float32


Epoch 1/10:   2%|▏         | 2/100 [00:00<00:28,  3.44it/s]

Input shape: torch.Size([16, 501, 64, 64]), Input type: torch.float32
Labels shape: torch.Size([16]), Labels type: torch.int64
Outputs shape: torch.Size([16, 26]), Outputs type: torch.float32


Epoch 1/10:   3%|▎         | 3/100 [00:00<00:26,  3.63it/s]

...

Outputs shape: torch.Size([16, 26]), Outputs type: torch.float32
Input shape: torch.Size([16, 501, 64, 64]), Input type: torch.float32
Labels shape: torch.Size([16]), Labels type: torch.int64
Outputs shape: torch.Size([16, 26]), Outputs type: torch.float32


Epoch 10/10:  98%|█████████▊| 98/100 [00:22<00:00,  4.97it/s]

Input shape: torch.Size([16, 501, 64, 64]), Input type: torch.float32
Labels shape: torch.Size([16]), Labels type: torch.int64
Outputs shape: torch.Size([16, 26]), Outputs type: torch.float32
Input shape: torch.Size([16, 501, 64, 64]), Input type: torch.float32
Labels shape: torch.Size([16]), Labels type: torch.int64


Epoch 10/10: 100%|██████████| 100/100 [00:22<00:00,  4.46it/s]

Outputs shape: torch.Size([16, 26]), Outputs type: torch.float32
Input shape: torch.Size([6, 501, 64, 64]), Input type: torch.float32
Labels shape: torch.Size([6]), Labels type: torch.int64
Outputs shape: torch.Size([6, 26]), Outputs type: torch.float32


In [34]:
 # Evaluate the model on training data
''' 
    Args:
    train_accuracies.append(train_accuracy): appends accruacy of the model on train data
    test_accuracies.append(test_accuracy): appends the accuracy of the model on the test data
    train_aucs.append(train_auc): adds the train_auc to the train_aucs
    evaluate_model(train_loader): will return train_accuracy, train_auc, and train_loss
    test_accuracies.append(test_accuracy): adds test_accuracy to list of test_accuracies
    test_aucs.append(test_auc): appends test_auc to a list test_aucs
    test_losses.append(test_loss): appends test_loss to test_losses
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Train Acc: {train_accuracy}, 
    Train AUC: {train_auc}, Test Acc: {test_accuracy}, Test AUC: {test_auc}'): f string that contains variables noted above

    Return: outputs of train loss, test loss, train accuracy and test accuracy, train auc and test auc 

    
'''
train_accuracy, train_auc, train_loss = evaluate_model(train_loader)
train_accuracies.append(train_accuracy)
train_aucs.append(train_auc)
    
 # Evaluate the model on test dataset, test_loader
test_accuracy, test_auc, test_loss = evaluate_model(test_loader)
test_accuracies.append(test_accuracy)
test_aucs.append(test_auc)
test_losses.append(test_loss)
    
print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, '
        f'Train Acc: {train_accuracy}, Train AUC: {train_auc}, '
        f'Test Acc: {test_accuracy}, Test AUC: {test_auc}')

all_probs shape: (1590, 26)
all_labels shape: (1590,)
Sample all_probs: [array([9.9991548e-01, 2.4121452e-13, 1.4708215e-05, 2.7037788e-13,
       5.3513438e-09, 1.9680113e-09, 4.8000180e-12, 1.7747736e-07,
       8.1810732e-11, 3.2857113e-13, 2.0803029e-10, 6.1278966e-05,
       1.6374010e-11, 1.9215049e-10, 8.2654769e-06, 2.0179496e-17,
       4.6259333e-12, 1.2108382e-10, 1.4061541e-14, 6.2828964e-11,
       1.3934576e-08, 2.1662690e-14, 4.5419725e-13, 1.6855803e-14,
       8.2171229e-14, 3.9903352e-17], dtype=float32), array([9.9991548e-01, 2.4121452e-13, 1.4708215e-05, 2.7037788e-13,
       5.3513438e-09, 1.9680113e-09, 4.8000180e-12, 1.7747736e-07,
       8.1810732e-11, 3.2857113e-13, 2.0803029e-10, 6.1278966e-05,
       1.6374010e-11, 1.9215049e-10, 8.2654769e-06, 2.0179496e-17,
       4.6259333e-12, 1.2108382e-10, 1.4061541e-14, 6.2828964e-11,
       1.3934576e-08, 2.1662690e-14, 4.5419725e-13, 1.6855803e-14,
       8.2171229e-14, 3.9903352e-17], dtype=float32), array([2.494814

all_probs shape: (1532, 26)
all_labels shape: (1532,)
Sample all_probs: [array([8.6057753e-07, 1.4037112e-04, 4.2054574e-03, 8.7132001e-01,
       3.0594489e-05, 7.5703174e-05, 3.3588949e-05, 3.0351143e-02,
       6.6711744e-03, 7.5509869e-02, 2.6569248e-03, 7.4255331e-06,
       7.8610372e-04, 1.2634394e-08, 1.2963790e-09, 1.4386331e-05,
       1.3449518e-09, 1.4535293e-04, 9.2491542e-11, 6.5022703e-11,
       3.1869006e-03, 4.7812684e-11, 3.9332127e-04, 1.0573483e-11,
       4.3346941e-11, 4.4708601e-03], dtype=float32), array([2.8915045e-01, 2.8088607e-06, 6.9254327e-01, 2.7035949e-06,
       5.0291848e-05, 1.2428985e-02, 5.4481388e-05, 1.6633818e-03,
       1.1911559e-04, 6.8687797e-05, 5.6068508e-05, 3.0959307e-03,
       2.0962432e-06, 2.9012087e-08, 4.8629218e-04, 2.1619787e-08,
       3.3101438e-05, 9.3823273e-06, 3.7913523e-08, 3.5160801e-07,
       2.3147206e-04, 5.0519240e-08, 8.5099572e-07, 2.7584278e-08,
       6.2958250e-08, 9.4101409e-08], dtype=float32), array([6.313182

In [33]:
#Saving the model 
'''
    Args:
    model.state_dict(): contains all the learnable parameters and weights of the model
    Returns: saving the current state of the model's parameters
'''
torch.save(model.state_dict(), "SimpleCNNdivisions.pth")